# import

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import math
import glob
import re
import random
import collections
import os
import sys
import pickle

# data

In [3]:
botname = 'แว่นเป๋อ❄️'
in_folder = '/Documents/Chula/3/NLP/stanford-tensorflow-tutorials/assignments/chatbot/Wannta/Linechat'

In [114]:
import os
convs = []
for file in os.listdir(in_folder):
    if(file == 'out.txt'):
        continue
    with open(in_folder+file) as f:
        print(file)

        conv = []
        isFirst = True
        isMultipleLine = False
        is_human_first = True

        combined_sent = []
        prev_send = ''
        
        for line in f:            
            sender = ''
            sent = ''
            spl = line.strip().split('\t')
            
            if(len(spl) == 3):
                
                sender = spl[1]
                sent = spl[2]
                
                if(isFirst):
                    if(sender == botname):
                        is_human_first = False
                    isFirst = False

                if(sent in ['[Photo]','[Sticker]','[File]' , '[Video]']):
                    continue
                if(sent[0] == '☎'):
                    continue
                
                if(sent[0] == '"'):
                    sent = sent[1:]
                    isMultipleLine = True
                    
                if(sender != prev_send):
                    conv.append(' '.join(combined_sent))
                    combined_sent = [sent]
                    prev_send = sender
                    
                else:
                    combined_sent.append(sent)
            
            elif(isMultipleLine):
                
                combined_sent.append(line.strip().strip('"'))
                if(line[-1] == '"'):
                    isMultipleLine = False

                
#             print(combined_sent)

        combined_sent.append(sent)
        if( is_human_first ):
            conv = conv[1:]
        convs.append(conv)
        
with open(in_folder+'out.txt', 'w') as o:
    for con in convs:
        for sent in con:
            o.write(sent+'\n')

[LINE] Chat with KNT.txt
[LINE] Chat with OaT.txt
[LINE] Chat with mos.txt
[LINE] Chat with NoTT'z 🔩.txt
[LINE] Chat with Ten'ten.txt
LINE Chat with Jaja 2559 (1).txt
[LINE] Chat with Fair.txt


In [9]:
out_path = 'out/'

In [11]:
conver = []
with open(out_path+'out.txt') as f:
    for line in f:
        conver.append(line)

# tokenize

In [8]:
import deepcut
deepcut.tokenize("สวัสดีเพื่อนๆ")

Using TensorFlow backend.
/home/aroundy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


['สวัสดี', 'เพื่อน', 'ๆ']

In [14]:
%%time
token = []
for sent in conver:
    token.append(deepcut.tokenize(sent.strip()))
print(token[:5])

[[''], ['เฮ้ยู', ' ', 'SA', 'จะ', 'อยู่', 'กะ', 'ใคร', '555'], ['อยู่', 'กะ', 'บอสงับบ'], ['ทุก', 'คน', 'ไป', 'อยู่', 'กับ', 'บอส', '5555', ' ', 'โอเค', '555'], ['อ่าวว', ' ', 'ละ', 'มี', 'คน', 'อยู่', 'ป่า', 'วว']]
CPU times: user 4min 3s, sys: 3.98 s, total: 4min 7s
Wall time: 1min 17s


In [28]:
token_set = set()
for words in token:
    token_set = token_set.union(words)

token_list = list(token_set)
print(token_list)

['', 'theta', 'ฮีปอ่ะ', 'ไฟล์', 'numpy', 'เมื่อ', 'ปะละ', 'หุยยยยยยยยยยยยยย', 'Type', '40000', 'โค๊ด', 'อะนะะ', 'โครต', 'ตัด', 'ขึ้น', 'แม้ต', 'อ๊าว', '{', 'รันซ้ำ', 'จ๋าาาาาา', 'จริงใจ', 'ภาย', 'S.sname FROM', 'ป้า', 'โอชิกี่', 'กลับ', 'ต่อ', 'ปัญหา', 'เคส', 'งื๊อ', 'อาจารย์', 'ฟิซิม', 'จำ', 'อีป็อก', 'layer', 'กิ๊ก', 'ปีนะ', 'พัง', 'เมนมา', 'implement', 'อัพเด', 'สรุป', 'ใบ้', 'สติกเกอร์', 'แพลน', 'tf', 'หืม', 'Pick', 'รีวว', 'ถึงยัง', 'อยู่', 'เซียน', 'ดิ้นหรอ', 'เนี่ย', 'ดาต้าเบส', 'บอสมะ', 'เยอะ', 'ชดเชย สินะ', 'เอวี', 'ท้าย', 'เมลไร', 'If', 'You', 'เจอะ', 'อิ้อิ้', 'บาย', 'ห้องน้ำ', 'อย่า', 'แปป', 'ยากจัง', 'แคปสกรีน', 'มาเจอ', 'ปปปป', 'หิววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววววว

In [16]:
def create_dataset_dict(input_list):
    words = []
    for review in input_list:
        for word in review:
            words.append(word)
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    
    #threshold = len(input_text)//1e6
    threshold = 3
    print("Threshold", threshold)
    
    word_unk = set()
    num_unk = 0
    idx = len(word_count)-1
    while word_count[idx][1] <= threshold:
        word_unk.add(word_count[idx][0])
        num_unk += word_count[idx][1]
        idx -= 1

    word_count = word_count[:idx+1]
    
    #include a token for unknown word
    word_count.append(("UNK",num_unk))
    word_count = sorted(word_count, key=lambda x: -x[1])
    #word_count = [x for x in word_count if x[0] not in word_unk]
    
    #print out 10 most frequent words
    print(word_count[:10])
    print("#unk" ,num_unk)
    
    dictionary = dict()
    dictionary["for_keras_zero_padding"] = 0
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    datas = list()
    for review in input_list:
        data = []
        for word in review:
            if word in word_unk:
                data.append(dictionary["UNK"])
            else:
                data.append(dictionary[word])
        datas.append(data)

    return datas,dictionary, reverse_dictionary

In [31]:
dataset,dictionary, reverse_dictionary=create_dataset_dict(token)

Threshold 3
[(' ', 9373), ('UNK', 5619), ('ไม่', 1033), ('ไป', 820), ('ได้', 731), ('จะ', 654), ('ๆ', 635), ('ทำ', 518), ('ก็', 495), ('มัน', 488)]
#unk 5619


In [34]:
print("len data set:", len(dataset))
print("len dictionary:", len(dictionary))
print("output sample (dataset):",dataset[:10])
print("output sample (dictionary):",{k: dictionary[k] for k in list(dictionary)[:10]}, "\ndictionary size: ",len(dictionary))
print("output sample (reverse dictionary):",{k: reverse_dictionary[k] for k in list(reverse_dictionary)[:10]})

len data set: 3385
len dictionary: 1075
output sample (dataset): [[779], [779, 1, 349, 6, 32, 220, 179, 15], [32, 220, 779], [152, 48, 4, 32, 102, 782, 49, 1, 318, 15], [779, 1, 12, 13, 48, 32, 779, 303], [13, 7, 15], [779, 32, 190, 85], [35, 18, 283, 48], [16, 120, 319, 7, 21, 4, 782, 1, 779, 1, 283, 1, 75], [320, 783]]
output sample (dictionary): {'for_keras_zero_padding': 0, ' ': 1, 'UNK': 779, 'ไม่': 3, 'ไป': 4, 'ได้': 5, 'จะ': 6, 'ๆ': 7, 'ทำ': 8, 'ก็': 9} 
dictionary size:  1075
output sample (reverse dictionary): {0: 'for_keras_zero_padding', 1: ' ', 779: '172', 3: 'ไม่', 4: 'ไป', 5: 'ได้', 6: 'จะ', 7: 'ๆ', 8: 'ทำ', 9: 'ก็'}


In [48]:
human = []
bot = []
even = True
for line in dataset:
    if (even):
        human.append(line)
        even = False
    else:
        bot.append(line)
        even = True
human_train = human[int(len(human)*0.7):]
human_test = human[:int(len(human)*0.7)]
bot_train = bot[int(len(bot)*0.7):]
bot_test = bot[:int(len(bot)*0.7)]
with open(out_path+'human_train.txt', 'w') as o:
    for line in human_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_train.txt', 'w') as o:
    for line in bot_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'human_test.txt', 'w') as o:
    for line in human_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_test.txt', 'w') as o:
    for line in bot_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')

In [47]:
dict_list = []
with open(out_path+'vocab.txt','w') as o:
    for value in dictionary:
        o.write(value+'\n')